In [39]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import sys
import pdb
import numpy as np
import re
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)


In [40]:
os.chdir('/home2/s439906/project/CASP16/monomer/')
# get raw score
def get_raw_score(stage, group_name):
    if stage == "1":
        score_path = "./monomer_data_Sep_15_EU/raw_data/"
    elif stage == "0":
        score_path = "./monomer_data_T0_EU/raw_data/"
    elif stage == "2":
        score_path = "./monomer_data_T2_EU/raw_data/"
    # score_path = "./monomer_data_Sep_15_EU/raw_data/"
    files = [csv for csv in os.listdir(score_path) if csv.endswith(".csv")]
    # sort the files
    files.sort()
    # group_name = "241"
    measures = ['GDT_HA', 'GDC_SC', 'AL0_P', 'SphGr', 'CAD_AA', 'QSE', 'LDDT', 'MolPrb_Score', 'reLLG_const']
    TS_group_name = "TS" + group_name
    all_df = None
    for file in files:
        data = pd.read_csv(score_path + file, index_col=0)
        # get the measure columns
        measure_data = data[measures]
        # get rows where index contains TS_group_name
        TS_data = measure_data[measure_data.index.str.contains(TS_group_name)]
        if all_df is None:
            all_df = TS_data
        else:
            all_df = pd.concat([all_df, TS_data])
    # print columns that has T1228*D1 pattern
    # 查找符合 T1228*D1 模式的列
    if stage == "1":
        remaining_df = all_df.copy()
        list_1 = [fr'T1228v1.*_{n}-D1' for n in range(1, 6)]
        list_2 = [fr'T1228v2.*_{n}-D1' for n in range(1, 6)]
        list_3 = [fr'T1228v1.*_{n}-D2' for n in range(1, 6)]
        list_4 = [fr'T1228v2.*_{n}-D2' for n in range(1, 6)]
        list_5 = [fr'T1228v1.*_{n}-D3' for n in range(1, 6)]
        list_6 = [fr'T1228v2.*_{n}-D3' for n in range(1, 6)]
        list_7 = [fr'T1228v1.*_{n}-D4' for n in range(1, 6)]
        list_8 = [fr'T1228v2.*_{n}-D4' for n in range(1, 6)]
        list_9 = [fr'T1239v1.*_{n}-D1' for n in range(1, 6)]
        list_10 = [fr'T1239v1.*_{n}-D2' for n in range(1, 6)]
        list_11 = [fr'T1239v1.*_{n}-D3' for n in range(1, 6)]
        list_12 = [fr'T1239v1.*_{n}-D4' for n in range(1, 6)]   
        list_13 = [fr'T1294v1.*_{n}-D1' for n in range(1, 6)]
        list_14 = [fr'T1294v2.*_{n}-D1' for n in range(1, 6)]
        # get the list of all patterns
        list_all = list_1 + list_2 + list_3 + list_4 + list_5 + list_6 + list_7 + list_8 + list_9 + list_10 + list_11 + list_12 + list_13 + list_14
        for p in list_all:
            pattern = fr'{p}'
            matching_rows = remaining_df[remaining_df.index.str.match(pattern)].sort_index()
            remaining_df = remaining_df[~remaining_df.index.str.match(pattern)]
            # 对于提取出的部分，取每一行的最大值
            max_values = matching_rows.max(axis=0)
            # print(max_values)
            # 使用提取部分的第一个 index 作为新行的名称，确保列名唯一
            if not matching_rows.empty:
                new_row_name = f"{matching_rows.index[0]}"
                remaining_df.loc[new_row_name] = max_values
                
        all_df = remaining_df
    # re sort the index
    all_df = all_df.sort_index()
    return all_df

if not os.path.exists("for_Arne"):
    os.makedirs("for_Arne")
for stage in ["0", "1", "2"]:
    for group_name in ["241", "304"]:
        all_df = get_raw_score(stage, group_name)
        # save 4 digit precision
        # all_df.to_csv("for_Arne/" + "TS_" + group_name + "_stage_" + stage + "_raw.csv")
        # all_df = all_df.round(4)
        all_df = all_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
        all_df.to_csv("for_Arne/" + "TS_" + group_name + "_stage_" + stage + "_raw.csv", float_format='%.3f')


for stage in ["0", "1", "2"]:
    for group_name in ["274", "494"]:
        all_df = get_raw_score(stage, group_name)
        # save 4 digit precision
        all_df = all_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
        all_df.to_csv("for_Dima/" + "TS_" + group_name + "_stage_" + stage + "_raw.csv", float_format='%.4f')

    
# data_test = pd.read_csv("for_Arne/TS_241_stage_1_raw.csv", index_col=0)
# data_test.head(100)

In [ ]:
os.chdir('/home2/s439906/project/CASP16/monomer/')

def get_z_score(stage, group_name):
    # measures = ['GDT_HA', 'GDC_SC', 'AL0_P', 'SphGr', 'CAD_AA', 'QSE', 'LDDT', 'MolPrb_Score', 'reLLG_const']
    
    measures = [
                        'GDT_HA', 'GDC_SC', 'reLLG_const',
                        'QSE',
                        'AL0_P', 'SphGr',
                        'CAD_AA', 'LDDT',
                        'MolPrb_Score',
                    ]
    weights = [
                        1/6, 1/16, 1/6,
                        1/6,
                        1/16, 1/8,
                        1/8, 1/16,
                        1/16,
                    ]
    # model = "first"
    model = "best"
    # mode = "easy"
    # mode = "medium"
    # mode = "hard"
    mode = "all"
    impute_value = -2
    if stage == "1":
        score_path = "./group_by_target_EU/"
    elif stage == "0":
        score_path = "./score_T0/"
    elif stage == "2":
        score_path = "./score_T2/"
    TS_group_name = "TS" + group_name
    T1_df = None 
    for measure in measures:
        score_file = f"{measure}-{model}-{mode}-impute={impute_value}-sum_unweighted.csv"
        score = pd.read_csv(score_path + score_file, index_col=0)
        # get the rows where index contains TS_group_name
        score = score[score.index.str.contains(TS_group_name)]
        # print(score)
        if T1_df is None:
            T1_df = score
        else:
            T1_df = pd.concat([T1_df, score])
    # if there is a column called sum, remove it
    try:
        T1_df = T1_df.drop(columns=['sum'])
        
    except:
        pass
    # print(T1_df)
    T1_df=T1_df.T
    # change the column names to measure names
    T1_df.columns = measures
    # compute weighted sum
    T1_df['weighted_sum'] = T1_df[measures].dot(weights)
    return T1_df
for stage in ["0", "1", "2"]:
    for group_name in ["241", "304"]:
        T1_df = get_z_score(stage, group_name)
        T1_df = T1_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
        
        T1_df.to_csv("for_Arne/" + "TS_" + group_name + "_stage_" + stage + "_z_score.csv", float_format='%.4f')
        # print(T1_df)
# print(T1_df)

for stage in ["0", "1", "2"]:
    for group_name in ["274", "494"]:
        T1_df = get_z_score(stage, group_name)
        T1_df = T1_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
        T1_df.to_csv("for_Dima/" + "TS_" + group_name + "_stage_" + stage + "_z_score.csv", float_format='%.4f')
        # print(T1_df)

In [ ]:
os.chdir('/home2/s439906/project/CASP16/oligomer/')
def get_raw_score(stage, group_name):
    if not os.path.exists("for_Arne"):
        os.makedirs("for_Arne")
    TS_group_name = "TS" + group_name
    if stage == "1":
        score_path = "./score_interface_v2_np/"
    elif stage == "0":
        score_path = "./group_by_target_per_interface_T0/"
    elif stage == "2":
        score_path = "./score_interface_v2_T2/"
    
    measures = ['dockq_mean_inclNone', 'qs_best_mean_inclNone',
                #  'qs_global_mean_inclNone',
                'ics_mean_inclNone', 'ips_mean_inclNone']

        # model = "first"
    model = "best"
        # mode = "easy"
        # mode = "medium"
        # mode = "hard"
    mode = "all"
    impute_value = -2
    score_df = None
    for measure in measures:
        score_file = f"{measure}-per_model.csv"
        score = pd.read_csv(score_path + score_file, index_col=0)
        # get the rows where index contains TS_group_name
        score = score[score.index.str.contains(TS_group_name)]
        if score_df is None:
            score_df = score
        else:
            score_df = pd.concat([score_df, score], axis=1)
    print(score_df)

    measures = ['lddt', 'tm_score']
    if stage == "1":
        score_path = "./score_EU_v2_np/"
    elif stage == "0":
        score_path = "./group_by_target_EU_new_T0/"
    elif stage == "2":
        score_path = "./score_EU_v2_T2/"
    score_df_2 = None
    for measure in measures:
        score_file = f"{measure}-{model}-{mode}-impute={impute_value}_per_model.csv"
        score = pd.read_csv(score_path + score_file, index_col=0)
        # get the rows where index contains TS_group_name
        score = score[score.index.str.contains(TS_group_name)]
        if score_df_2 is None:
            score_df_2 = score
        else:
            score_df_2 = pd.concat([score_df_2, score], axis=1)
    score_df_2
    # print(score_df_2)

    # merge the dataframes
    merged_df = pd.concat([score_df, score_df_2], axis=1)
    merged_df
    measures = ["DockQ", "QSbest", "ICS", "IPS", "lDDT", "TM-score"]
    merged_df.columns = measures
    # set the index name to be Models
    merged_df.index.name = "Model"

    if stage == "1":
        remaining_df = merged_df.copy()
        list_1 = [fr'T1249v1.*_{n}o' for n in range(1, 6)]
        list_2 = [fr'T1249v2.*_{n}o' for n in range(1, 6)]
        # get the list of all patterns
        list_all = list_1 + list_2
        for p in list_all:
            pattern = fr'{p}'
            matching_rows = remaining_df[remaining_df.index.str.match(pattern)].sort_index()
            remaining_df = remaining_df[~remaining_df.index.str.match(pattern)]
            # 对于提取出的部分，取每一行的最大值
            max_values = matching_rows.max(axis=0)
            # print(max_values)
            # 使用提取部分的第一个 index 作为新行的名称，确保列名唯一
            if not matching_rows.empty:
                new_row_name = f"{matching_rows.index[0]}"
                remaining_df.loc[new_row_name] = max_values
                
        merged_df = remaining_df
    # re sort the index
    merged_df = merged_df.sort_index()
    return merged_df
if not os.path.exists("for_Arne"):
    os.makedirs("for_Arne")
for stage in ["0", "1", "2"]:
    for group_name in ["241", "304"]:
        all_df = get_raw_score(stage, group_name)
        all_df = all_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)

        all_df.to_csv("for_Arne/" + "TS_" + group_name + "_stage_" + stage + "_raw.csv", float_format='%.4f')
if not os.path.exists("for_Dima"):
    os.makedirs("for_Dima")
for stage in ["0", "1", "2"]:
    for group_name in ["274", "494"]:
        all_df = get_raw_score(stage, group_name)
        all_df = all_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
        all_df.to_csv("for_Dima/" + "TS_" + group_name + "_stage_" + stage + "_raw.csv", float_format='%.4f')

               dockq_mean_inclNone  qs_best_mean_inclNone  ics_mean_inclNone  ips_mean_inclNone
model                                                                                          
H0208TS241_1              0.814000               0.865000           0.818000           0.827000
H0208TS241_2              0.818000               0.871000           0.804000           0.833000
H0208TS241_3              0.821000               0.868000           0.814000           0.833000
H0208TS241_4              0.809000               0.866000           0.796000           0.840000
H0208TS241_5              0.822000               0.871000           0.810000           0.825000
...                            ...                    ...                ...                ...
T0270TS241_1o             0.567961               0.715147           0.661394           0.676614
T0270TS241_2o             0.546843               0.708136           0.651267           0.655620
T0270TS241_3o             0.527705      

In [43]:
os.chdir('/home2/s439906/project/CASP16/oligomer/')
def get_z_score(stage, group_name):
    measures = ['dockq_mean_inclNone', 'qs_best_mean_inclNone',
                #  'qs_global_mean_inclNone',
                'ics_mean_inclNone', 'ips_mean_inclNone']
    TS_group_name = "TS" + group_name
    if stage == "1":
        score_path = "./score_interface_v2_np/"
    elif stage == "0":
        score_path = "./group_by_target_per_interface_T0/"
    elif stage == "2":
        score_path = "./score_interface_v2_T2/"
    model = "best"
        # mode = "easy"
        # mode = "medium"
        # mode = "hard"
    mode = "all"
    impute_value = -2
    T1_df = None   
    for measure in measures:
        score_file = f"{measure}-{model}-{mode}-impute={impute_value}_unweighted_EU.csv"
        score = pd.read_csv(score_path + score_file, index_col=0)
        # get the rows where index contains TS_group_name
        score = score[score.index.str.contains(TS_group_name)]
        # print(score)
        if T1_df is None:
            T1_df = score
        else:
            T1_df = pd.concat([T1_df, score])
    # if there is a column called sum, remove it
    T1_df = T1_df.drop(columns=['sum'])
    T1_df=T1_df.T
    # change the column names to measure names
    T1_df.columns = measures
    T1_df
    # print(T1_df)

    measures = ['lddt', 'tm_score']
    if stage == "1":
        score_path = "./score_EU_v2_np/"
    elif stage == "0":
        score_path = "./group_by_target_EU_new_T0/"
    elif stage == "2":
        score_path = "./score_EU_v2_T2/"
    T1_df_2 = None   
    for measure in measures:
        score_file = f"{measure}-{model}-{mode}-impute={impute_value}_sum.csv"
        score = pd.read_csv(score_path + score_file, index_col=0)
        # get the rows where index contains TS_group_name
        score = score[score.index.str.contains(TS_group_name)]
        # print(score)
        if T1_df_2 is None:
            T1_df_2 = score
        else:
            T1_df_2 = pd.concat([T1_df_2, score])
    # if there is a column called sum, remove it
    T1_df_2 = T1_df_2.drop(columns=['sum'])
    T1_df_2=T1_df_2.T
    # change the column names to measure names
    T1_df_2.columns = measures
    T1_df_2
    # merge the dataframes
    merged_df = pd.concat([T1_df, T1_df_2], axis=1)
    weights = [
        1/6, 1/6, 1/6, 1/6, 1/6, 1/6
    ]
    print(merged_df.shape)
    measures = ['dockq_mean_inclNone', 'qs_best_mean_inclNone',
                'ics_mean_inclNone', 'ips_mean_inclNone', 'lddt', 'tm_score']
    merged_df['weighted_sum'] = merged_df[measures].dot(weights)
    return merged_df
if not os.path.exists("for_Arne"):
    os.makedirs("for_Arne")
for stage in ["0", "1", "2"]:
    for group_name in ["241", "304"]:
        T1_df = get_z_score(stage, group_name)
        T1_df = T1_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
        T1_df.to_csv("for_Arne/" + "TS_" + group_name + "_stage_" + stage + "_z_score.csv", float_format='%.4f')
        # print(T1_df)
if not os.path.exists("for_Dima"):
    os.makedirs("for_Dima")
for stage in ["0", "1", "2"]:
    for group_name in ["274", "494"]:
        T1_df = get_z_score(stage, group_name)
        T1_df = T1_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)
        T1_df.to_csv("for_Dima/" + "TS_" + group_name + "_stage_" + stage + "_z_score.csv", float_format='%.4f')
        # print(T1_df)

(28, 6)
(28, 6)
(40, 6)
(40, 6)
(34, 6)
(34, 6)
(28, 6)
(28, 6)
(40, 6)
(40, 6)
(34, 6)
(34, 6)
